In [1]:
import jax
import jaxlib

print("jax version:", jax.__version__)
print("jaxlib version:", jaxlib.__version__)

import scvi
import h5py
import anndata as ad
import pandas as pd
import scipy.sparse as sp
import numpy as np

jax version: 0.4.34
jaxlib version: 0.4.34


/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_atse_counts' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_junc_counts' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [7]:
combined_adata = ad.read_h5ad('/Users/smriti/Documents/Research_Knowles_Lab/multivi_tools_splicing/ann_data/TMS_BRAINONLY_Combined_GE_ATSE.h5ad')
print(combined_adata.X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 92249252 stored elements and shape (19942, 77042)>
  Coords	Values
  (0, 1)	3.0
  (0, 31)	894.0
  (0, 33)	14.0
  (0, 41)	375.0
  (0, 47)	232.0
  (0, 53)	4.0
  (0, 62)	126.0
  (0, 81)	422.0
  (0, 83)	316.0
  (0, 123)	689.0
  (0, 253)	1.0
  (0, 373)	9.0
  (0, 584)	110.0
  (0, 589)	204.0
  (0, 607)	146.0
  (0, 620)	1.0
  (0, 647)	314.0
  (0, 704)	1.0
  (0, 708)	55.0
  (0, 723)	384.0
  (0, 758)	136.0
  (0, 761)	63.0
  (0, 765)	109.0
  (0, 804)	265.0
  (0, 815)	98.0
  :	:
  (19941, 76361)	-0.02289545657822932
  (19941, 76362)	-0.01758125984632416
  (19941, 76363)	0.1206251333044655
  (19941, 76364)	-0.014234563710877354
  (19941, 76365)	-0.20714516825665372
  (19941, 76628)	-0.23058531115119138
  (19941, 76629)	-0.10831592598971826
  (19941, 76630)	0.33890123714093257
  (19941, 76694)	-0.021787400161015828
  (19941, 76695)	0.02178740016101377
  (19941, 76784)	-0.011140687378338374
  (19941, 76785)	-0.3193632429020339
  (19941, 76

In [3]:
import torch

In [4]:
scvi.model.MULTIVISPLICE.setup_anndata(combined_adata, batch_key="modality", atse_counts='cell_by_cluster_matrix', junc_counts='cell_by_junction_matrix')

In [5]:

model = scvi.model.MULTIVISPLICE(
    combined_adata,
    n_genes=(combined_adata.var["modality"] == "Gene_Expression").sum(),
    n_junctions=(combined_adata.var["modality"] == "Splicing").sum(),
)

model.view_anndata_setup()

/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Anndata setup with scvi-tools version 1.2.0.

Setup via `MULTIVISPLICE.setup_anndata` with arguments:

{
│   'layer': None,
│   'atse_counts': 'cell_by_cluster_matrix',
│   'junc_counts': 'cell_by_junction_matrix',
│   'batch_key': 'modality',
│   'size_factor_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None,
│   'protein_expression_obsm_key': None,
│   'protein_names_uns_key': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│    n_atse_counts_key     │ 77042 │
│         n_batch          │   1   │
│         n_cells          │ 19942 │
│ n_extra_categorical_covs │   0   │
│ n_extra_continuous_covs  │   0   │
│    n_junc_counts_key     │ 77042 │
│         n_labels         │   1   │
│          n_vars          │ 77042 │
└──────────────────────────┴───────┘

                        Data Registry                        
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Registry Key   ┃           scvi-tools Location           ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        X        │                 adata.X                 │
│ atse_counts_key │ adata.layers['cell_by_cluster_matrix']  │
│      batch      │        adata.obs['_scvi_batch']         │
│      ind_x      │          adata.obs['_indices']          │
│ junc_counts_key │ adata.layers['cell_by_junction_matrix'] │
│     labels      │        adata.obs['_scvi_labels']        │
└─────────────────┴─────────────────────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

In [6]:
model.train()

/var/folders/kh/_2z63fg956v84_0hgyj8k3y00000gn/T/ipykernel_78081/1676823408.py:1: DeprecationWarning: `save_best` is deprecated in v1.2 and will be removed in v1.3. Please use `enable_checkpointing` instead. See https://github.com/scverse/scvi-tools/issues/2568 for more details.
  model.train()
/Users/smriti/Documents/Research_Knowles_Lab/scvi-tools-splicing/src/scvi/model/_multivi_splice.py:343: UserWarning: Disabling adversarial classifier.
  training_plan = self._training_plan_cls(self.module, **plan_kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/smriti/opt/anaconda3/envs/scvi-en

Training:   0%|          | 0/500 [00:00<?, ?it/s]

shape: torch.Size([128, 15])
qz_v_info: tensor([ 0.0908, -0.1507,  0.4766, -0.3718,  0.0549, -0.1802,  0.1688,  0.1316,
         0.7538, -0.3140,  0.5400, -0.1575, -0.0520, -0.0297, -0.0207],
       grad_fn=<MinBackward0>)
qzv_spl_info: tensor([0.5397, 0.3101, 0.6066, 0.3894, 0.2942, 0.3895, 0.3647, 0.3209, 0.3308,
        0.6425, 0.6333, 0.3996, 0.4793, 0.6223, 0.4244],
       grad_fn=<MinBackward0>)
qzv_expr_info: tensor([0.5887, 0.3420, 0.4890, 0.3570, 0.5339, 0.4233, 0.5009, 0.4656, 1.0380,
        0.4374, 0.6417, 0.4431, 0.5263, 0.4221, 0.3280],
       grad_fn=<MinBackward0>)


ValueError: Expected parameter scale (Tensor of shape (128, 15)) of distribution Normal(loc: torch.Size([128, 15]), scale: torch.Size([128, 15])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[1.0922, 0.3604, 0.8172,  ..., 1.0499, 1.1047, 0.4597],
        [0.8132, 0.3867, 0.9177,  ..., 0.7437, 0.8666, 0.8992],
        [0.9462, 0.5543, 0.9436,  ..., 0.8629, 0.4028, 0.7507],
        ...,
        [0.8633, 0.2365, 0.9104,  ..., 1.1603, 0.6630, 0.5310],
        [1.0838, 0.5975, 1.1288,  ..., 0.6834, 0.9789, 0.9111],
        [0.6658, 0.7016, 1.0158,  ..., 0.8305, 0.8168, 0.6988]],
       grad_fn=<SqrtBackward0>)